In [1]:
# connecting libraries
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
import warnings
import re
warnings.filterwarnings('ignore')

In [4]:
# data loading

# data.mail data
# data_train = pd.read_csv('products_sentiment_train.tsv',sep='\t',header=None)
# data_test = pd.read_csv('products_sentiment_test.tsv',sep='\t')

# AMAZON data
data_train = pd.read_csv('train.ft.txt',sep='\t',header=None, nrows=10000)
data_test = pd.read_csv('products_sentiment_train.tsv',sep='\t',header=None)

# data_train[0] = data_train[0].map(lambda x: re.sub('[^a-zA-Z ]','', x.lower()))
# data_train['len'] = data_train[0].map(lambda x: len(x))
# data_test['text'] = data_test['text'].map(lambda x: re.sub('[^a-zA-Z ]','', x.lower()))
# data_test['len'] = data_test['text'].map(lambda x: len(x))

In [5]:
# preprocessing data

data_train['label'] = data_train[0].apply(lambda x: int(x[9]))
# rename
data_train.rename(columns={0:'text'},inplace=True)
data_test.rename(columns={0:'text',1:'label'},inplace=True)

data_train['text'] = data_train['text'].apply(lambda x: x[11:].lower())
data_test['text'] = data_test['text'].apply(lambda x: x.lower())
data_train.loc[data_train['label']==1,'label']=0
data_train.loc[data_train['label']==2,'label']=1
print(data_test.head())

                                                text  label
0          2 . take around 10,000 640x480 pictures .      1
1  i downloaded a trial version of computer assoc...      1
2  the wrt54g plus the hga7t is a perfect solutio...      1
3  i dont especially like how music files are uns...      0
4  i was using the cheapie pail ... and it worked...      1


In [65]:
# testing logistic_regression

tfidf_vec = TfidfVectorizer()
X = tfidf_vec.fit_transform(data_train['text'])
y = data_train['label'].values
CV = KFold(n_splits=5,shuffle=True,random_state=241)
# np.power(10,np.linspace(-5,5,11))
# np.linspace(0.01,10,20)
for C in np.linspace(1,10,5):
    log_los_values = []
    for train_index, test_index in CV.split(X):
        lr = LogisticRegression(penalty='l2',C=C)
        lr.fit(X[train_index],y[train_index])
        predict = lr.predict_proba(X[test_index])
        log_los_values.append(log_loss(y[test_index],predict[:,1]))
        #print(cv_results['test_score'])
    print(C, np.mean(log_los_values))
# 2.511886431509581 0.4820568735263769
# 2.323232323232323 0.48158033118945165
# 5.474747474747475 0.46380617530954443
# 5.451451451451451 0.463805939803994
# 5.4535353535353535 0.4638059371066535
# 5.929292929292929 0.4621232668547865

1.0 0.2760153003384427
6.2105263157894735 0.25460503755539526
11.421052631578947 0.25989785948337196
16.63157894736842 0.2667963624558804
21.842105263157894 0.273750614919507
27.052631578947366 0.2804565417444491


KeyboardInterrupt: 

In [71]:
# create result logistic_regression

tfidf_vec = TfidfVectorizer()
X_train = tfidf_vec.fit_transform(data_train['text'])
y_train = data_train['label'].values
lr = LogisticRegression(penalty='l2',C=5.4535353535353535)
lr.fit(X_train,y_train)
X_test = tfidf_vec.transform(data_test['text'])
predict = lr.predict_proba(X_test)

log_loss(data_test['label'],predict[:,1])

# data_test['y'] = predict[:,1]
# data_test.drop(columns='text',inplace=True)
# data_test.to_csv('result_lr.csv',index=False)

0.5390215732075905

In [7]:
# create result
tfidf_vec = TfidfVectorizer()
X_train = tfidf_vec.fit_transform(data_train[0])
# print(X_train[:10,...])
y_train = data_train[1].values
lr = LogisticRegression(penalty='l2',C=5.4535353535353535)
lr.fit(X_train,y_train)
X_test = tfidf_vec.transform(data_test['text'])
predict = lr.predict_proba(X_test)
data_test['y'] = predict[:,1]
data_test.drop(columns='text',inplace=True)
data_test.to_csv('result_lr.csv',index=False)

In [ ]:
# testing SVC

tfidf_vec = TfidfVectorizer()
X = tfidf_vec.fit_transform(data_train['text'])
y = data_train['label'].values
CV = KFold(n_splits=5,shuffle=True,random_state=241)
# np.power(10,np.linspace(-5,5,11))
# np.linspace(0.01,10,20)
for gamma in np.linspace(0.1,5,10):
    log_los_values = []
    for train_index, test_index in CV.split(X):
        svc = svm.SVC(gamma=gamma,probability=True)
        svc.fit(X[train_index],y[train_index])
        predict = svc.predict_proba(X[test_index])
    log_los_values.append(log_loss(y[test_index],predict[:,1]))
    print(gamma, np.mean(log_los_values))


# tfidf_vec = TfidfVectorizer()
# X = tfidf_vec.fit_transform(data_train[0])
# y = data_train[1].values
# CV = KFold(n_splits=5,shuffle=True,random_state=241)
# # np.power(10,np.linspace(0,1,100))
# # np.linspace(0.01,10,20)
# # for ker in ['poly', 'rbf', 'sigmoid']:
# for ker in ['rbf']:
#     for gamma in np.linspace(0.1,5,20):
#         log_los_values = []
#         for train_index, test_index in CV.split(X):
#             svc = svm.SVC(kernel=ker,gamma=gamma,probability=True)
#             svc.fit(X[train_index],y[train_index])
#             predict = svc.predict_proba(X[test_index])
#     #         print(predict.shape)
#             log_los_values.append(log_loss(y[test_index],predict))
#         print(f'kernal={ker}',gamma, np.mean(log_los_values))
#     print()

0.1 0.358107657183621
0.6444444444444445 0.33464764547293524


In [249]:
tfidf_vec = TfidfVectorizer()
X = tfidf_vec.fit_transform(data_train[0])
y = data_train[1].values
lr = LogisticRegression(penalty='l2',C=5.4535353535353535)
lr.fit(X,y)
X_test = tfidf_vec.transform(data_test['text'])
predict = lr.predict_proba(X_test)
print(predict[:10])
data_test['y'] = predict[:,1]
data_test.drop(columns='text',inplace=True)
data_test.to_csv('result_lr.csv',index=False)

[[0.05494269 0.94505731]
 [0.73234271 0.26765729]
 [0.10999111 0.89000889]
 [0.29136253 0.70863747]
 [0.62902439 0.37097561]
 [0.46072725 0.53927275]
 [0.07783871 0.92216129]
 [0.74498778 0.25501222]
 [0.89255639 0.10744361]
 [0.0402188  0.9597812 ]]




  (0, 3387)	1
  (0, 182)	1
  (0, 2469)	1
  (1, 972)	1
  (1, 3547)	1
  (1, 3685)	1
  (1, 2274)	1
  (1, 637)	2
  (1, 197)	1
  (1, 1167)	1
  (1, 1241)	1
  (1, 129)	2
  (1, 141)	1
  (1, 1216)	1
  (1, 1629)	1
  (1, 1935)	1
  (1, 3834)	1
  (1, 2983)	1
  (1, 3376)	1
  (1, 99)	1
  (1, 2356)	1
  (1, 80)	1
  (2, 1629)	1
  (2, 3439)	2
  (2, 3880)	1
  :	:
  (8, 86)	1
  (8, 3166)	1
  (8, 2243)	1
  (8, 1510)	1
  (8, 1074)	1
  (8, 3331)	1
  (8, 142)	1
  (8, 2106)	1
  (9, 3439)	2
  (9, 1719)	1
  (9, 3894)	1
  (9, 2325)	1
  (9, 176)	1
  (9, 3663)	1
  (9, 2385)	1
  (9, 3488)	1
  (9, 3458)	1
  (9, 3796)	1
  (9, 932)	1
  (9, 567)	1
  (9, 885)	1
  (9, 3557)	1
  (9, 2300)	1
  (9, 884)	1
  (9, 1382)	1
1e-05 0.6931471805599452
0.0001 0.6931471805599452
0.001 0.6931471805599452
0.01 0.6430126084890624
0.1 0.5208610732583445
1.0 0.8359718383331719
10.0 1.2614677996276922
100.0 1.4919501610812391
1000.0 1.6880817863297062
10000.0 1.8880640959562336
100000.0 1.960423630978972


In [50]:
# create result all
tfidf_vec = TfidfVectorizer()

X_train = data_train[0].values[:,np.newaxis]
train_size = X_train.shape[0]
X_test = data_test['text'].values[:,np.newaxis]
X_all = np.vstack((X_train,X_test))
X_all = pd.DataFrame(X_all)

X_all = tfidf_vec.fit_transform(X_all[0])
X_train = X_all[:train_size]
X_test = X_all[train_size:]
# print(X_train[:10,...])
y_train = data_train[1].values
lr = LogisticRegression(penalty='l2',C=5.516326530612245)
lr.fit(X_train,y_train)
predict = lr.predict_proba(X_test)
data_test['y'] = predict[:,1]
data_test.drop(columns='text',inplace=True)
data_test[['Id','y']].to_csv('result_lr.csv',index=False)

In [45]:
# testing all
print('\n')
tfidf_vec = TfidfVectorizer()
X_train = data_train[0].values[:,np.newaxis]
train_size = X_train.shape[0]
X_test = data_test['text'].values[:,np.newaxis]
X_all = np.vstack((X_train,X_test))
X_all = pd.DataFrame(X_all)
X_all = tfidf_vec.fit_transform(X_all[0])
X = X_all[:train_size]

y = data_train[1].values
print(X.shape,y.shape)
CV = KFold(n_splits=5,shuffle=True,random_state=241)
# np.power(10,np.linspace(-5,5,11))
# np.linspace(0.1,100,20)
for C in np.linspace(5.2,5.7,50):
    log_los_values = []
    for train_index, test_index in CV.split(X):
        lr = LogisticRegression(penalty='l2',C=C)
        lr.fit(X[train_index],y[train_index])
        predict = lr.predict_proba(X[test_index])
        log_los_values.append(log_loss(y[test_index],predict[:,1]))
        #print(cv_results['test_score'])
    print(C, np.mean(log_los_values))
# 2.511886431509581 0.4820568735263769
# 2.323232323232323 0.48158033118945165
# 5.474747474747475 0.46380617530954443
# 5.451451451451451 0.463805939803994
# 5.4535353535353535 0.4638059371066535



(2000, 4424) (2000,)
5.2 0.46336824282921363
5.210204081632654 0.4633645206966658
5.220408163265306 0.4633609832731086
5.2306122448979595 0.463357574385778
5.240816326530612 0.4633542932277409
5.251020408163265 0.4633511391025067
5.261224489795919 0.46334811149320243
5.271428571428571 0.46334520963785036
5.281632653061225 0.46334243266474295
5.291836734693877 0.46333978010136373
5.302040816326531 0.4633372511378565
5.312244897959184 0.4633348452872946
5.322448979591837 0.46333256165060793
5.33265306122449 0.4633303997358197
5.342857142857143 0.4633283587920607
5.353061224489796 0.4633264380752147
5.363265306122449 0.4633246372065664
5.373469387755102 0.46332295536783413
5.383673469387755 0.4633213918623019
5.393877551020409 0.4633199461480234
5.404081632653061 0.463318617620342
5.414285714285715 0.4633174055781262
5.424489795918367 0.4633163093249245
5.4346938775510205 0.46331468386520047
5.444897959183674 0.4633138136450429
5.455102040816326 0.46331305744635837
5.46530612244898 0.46

In [74]:
# testing boosting
tfidf_vec = TfidfVectorizer()
X = tfidf_vec.fit_transform(data_train[0])
y = data_train[1].values

# X = pd.DataFrame(X)
# print(X.head())

CV = KFold(n_splits=5,shuffle=True,random_state=241)
# np.power(10,np.linspace(0,1,100))
# np.linspace(0.01,10,20)
# for n_estimators in [150]:
for max_depth in [6]:
    for learning_rate  in [0.12, 0.1, 0.08, 0.06]:
        log_los_values = []
        for train_index, test_index in CV.split(X):
            gbc = GradientBoostingClassifier(learning_rate=learning_rate, max_depth=max_depth)
            gbc.fit(X[train_index],y[train_index])
            predict = gbc.predict_proba(X[test_index])
            log_los_values.append(log_loss(y[test_index],predict[:,1]))
        print(max_depth, f'learning_rate={learning_rate}', np.mean(log_los_values))

6 learning_rate=0.12 0.5247289270669494
6 learning_rate=0.1 0.5222220719833285
6 learning_rate=0.08 0.5290117317831526
6 learning_rate=0.06 0.5308788618672785


In [71]:
# test boosting all

tfidf_vec = TfidfVectorizer()
X_train = data_train[0].values[:,np.newaxis]
train_size = X_train.shape[0]
X_test = data_test['text'].values[:,np.newaxis]
X_all = np.vstack((X_train,X_test))
X_all = pd.DataFrame(X_all)
X_all = tfidf_vec.fit_transform(X_all[0])
X = X_all[:train_size]
y = data_train[1].values

CV = KFold(n_splits=5,shuffle=True,random_state=241)
# np.power(10,np.linspace(0,1,100))
# np.linspace(0.01,10,20)
# for n_estimators in [150]:
for max_depth in [4,5,6]:
    for learning_rate  in [0.2, 0.15, 0.1, 0.05]:
        log_los_values = []
        for train_index, test_index in CV.split(X):
            gbc = GradientBoostingClassifier(n_estimators=150, learning_rate=learning_rate, max_depth=max_depth)
            gbc.fit(X[train_index],y[train_index])
            predict = gbc.predict_proba(X[test_index])
            log_los_values.append(log_loss(y[test_index],predict[:,1]))
        print(max_depth, f'learning_rate={learning_rate}', np.mean(log_los_values))

4 learning_rate=0.2 0.5240379282728987
4 learning_rate=0.15 0.5213225539641342
4 learning_rate=0.1 0.5244499643281955
4 learning_rate=0.05 0.53806197522738
5 learning_rate=0.2 0.5298959235282794
5 learning_rate=0.15 0.5228579214573299
5 learning_rate=0.1 0.5219616173499515
5 learning_rate=0.05 0.5319201163167518
6 learning_rate=0.2 0.5455012410005685
6 learning_rate=0.15 0.531285834827399
6 learning_rate=0.1 0.5183754542228789
6 learning_rate=0.05 0.5265408237213677


In [35]:
# test lr & gbm
print('\n')
tfidf_vec = TfidfVectorizer()
X = tfidf_vec.fit_transform(data_train[0])
y = data_train[1].values
CV = KFold(n_splits=5,shuffle=True,random_state=241)
# np.power(10,np.linspace(0,1,100))
# np.linspace(0.01,10,20)


for my_const in [0.7,0.72,0.74,0.76,0.78,0.8]:
    for max_depth in [3,]:
        for learning_rate  in [0.1]:
            log_los_values = []
            for train_index, test_index in CV.split(X):
                lr = LogisticRegression(penalty='l2',C=5.4535353535353535)
                gbc = GradientBoostingClassifier(learning_rate=learning_rate, max_depth=max_depth)
                lr.fit(X[train_index],y[train_index])
                predict_lr = lr.predict_proba(X[test_index])[:,1]
                gbc.fit(X[train_index],y[train_index])
                predict_gbc = gbc.predict_proba(X[test_index])[:,1]
                predict = []
                for i in np.arange(len(predict_lr)):
                    if (predict_lr[i]>my_const) and (predict_gbc[i]>my_const):
                        predict.append(np.max([predict_lr[i],predict_gbc[i]]))
#                         predict.append(1)
                    else:
                        if (predict_lr[i]<1.0-my_const) and (predict_gbc[i]<1.0-my_const):
#                             predict.append(0)
                            predict.append(np.min([predict_lr[i],predict_gbc[i]]))
                        else:
                            predict.append(predict_lr[i]) 
                log_los_values.append(log_loss(y[test_index],predict))
                #print(cv_results['test_score'])
            print(f'my_const={my_const}',f'max_depth={max_depth}', f'log_loss={np.mean(log_los_values)}')
# my_const=0.78 max_depth=3 log_loss=0.46139022113954276



my_const=0.7 max_depth=3 log_loss=0.46209454434653663
my_const=0.72 max_depth=3 log_loss=0.46235528131905035
my_const=0.74 max_depth=3 log_loss=0.46187921995275183
my_const=0.76 max_depth=3 log_loss=0.4615832587575611
my_const=0.78 max_depth=3 log_loss=0.46139022113954276
my_const=0.8 max_depth=3 log_loss=0.46167967788689995


In [100]:
# create result lr & gbm
tfidf_vec = TfidfVectorizer()
X_train = tfidf_vec.fit_transform(data_train[0])
# print(X_train[:10,...])
y_train = data_train[1].values
X_test = tfidf_vec.transform(data_test['text'])
my_const=0.78

lr = LogisticRegression(penalty='l2',C=5.4535353535353535)
gbc = GradientBoostingClassifier()
lr.fit(X_train,y_train)
predict_lr = lr.predict_proba(X_test)[:,1]
gbc.fit(X_train,y_train)
predict_gbc = gbc.predict_proba(X_test)[:,1]
predict = []
for i in np.arange(len(predict_lr)):
    if (predict_lr[i]>my_const) and (predict_gbc[i]>my_const):
        predict.append(np.max([predict_lr[i],predict_gbc[i]]))
    else:
        if (predict_lr[i]<1.0-my_const) and (predict_gbc[i]<1.0-my_const):
            predict.append(np.min([predict_lr[i],predict_gbc[i]]))
        else:
            predict.append(predict_lr[i]) 



data_test['y'] = predict
data_test.drop(columns='text',inplace=True)
data_test.to_csv('result_lr.csv',index=False)

In [37]:
# testing SVC
print('\n')
tfidf_vec = TfidfVectorizer()
X = tfidf_vec.fit_transform(data_train[0])
y = data_train[1].values
CV = KFold(n_splits=5,shuffle=True,random_state=241)
# np.power(10,np.linspace(0,1,100))
# np.linspace(0.01,10,20)
# for ker in ['poly', 'rbf', 'sigmoid']:
for ker in ['rbf']:
    for gamma in np.linspace(0.1,5,20):
        log_los_values = []
        for train_index, test_index in CV.split(X):
            svc = svm.SVC(kernel=ker,gamma=gamma,probability=True)
            svc.fit(X[train_index],y[train_index])
            predict = svc.predict_proba(X[test_index])
    #         print(predict.shape)
            log_los_values.append(log_loss(y[test_index],predict))
        print(f'kernal={ker}',gamma, np.mean(log_los_values))
    print()

# kernal=rbf 1.7473684210526315 0.452176475069442
# kernal=rbf 1.9684210526315788 0.4520421615276179



kernal=rbf 0.1 0.47445741414027703
kernal=rbf 0.35789473684210527 0.4635631714270363
kernal=rbf 0.6157894736842106 0.46046140865244667
kernal=rbf 0.8736842105263158 0.45726590536156264
kernal=rbf 1.1315789473684212 0.4551298563466172
kernal=rbf 1.3894736842105266 0.45317260284083155
kernal=rbf 1.6473684210526318 0.45204670107772715
kernal=rbf 1.905263157894737 0.4520810842863046
kernal=rbf 2.1631578947368424 0.4526473661370642
kernal=rbf 2.421052631578948 0.45228601281545233
kernal=rbf 2.678947368421053 0.45429239768373675
kernal=rbf 2.936842105263158 0.4538788448852033
kernal=rbf 3.1947368421052635 0.45262386146546774
kernal=rbf 3.452631578947369 0.45320128481283517
kernal=rbf 3.710526315789474 0.4724836735693308
kernal=rbf 3.9684210526315793 0.4875705515449775
kernal=rbf 4.226315789473684 0.5159940410379049
kernal=rbf 4.484210526315789 0.5769910492059271
kernal=rbf 4.742105263157895 0.5934057152978102
kernal=rbf 5.0 0.6281182421315485



In [25]:
# testing SVC
from sklearn.svm import LinearSVC
tfidf_vec = TfidfVectorizer()
X = tfidf_vec.fit_transform(data_train[0])
y = data_train[1].values
CV = KFold(n_splits=5,shuffle=True,random_state=241)
# np.power(10,np.linspace(0,1,100))
# np.linspace(0.01,10,20)
for C in np.linspace(0.1,10,100):
    log_los_values = []
    for train_index, test_index in CV.split(X):
        svc = LinearSVC(C=C, max_iter=10000)
        svc.fit(X[train_index],y[train_index])
        predict = svc.decision_function(X[test_index])
        predict = (predict - predict.min()) / (predict.max() - predict.min())
        log_los_values.append(log_loss(y[test_index],predict))
    print(f'C={C}',np.mean(log_los_values))

C=0.1 0.5946426076814832
C=0.2 0.5885814471468649
C=0.30000000000000004 0.5882908917452914
C=0.4 0.5882718082307916
C=0.5 0.5882226871707538
C=0.6 0.5885091784490525
C=0.7000000000000001 0.5887329831778425
C=0.8 0.5894953627531114
C=0.9 0.5875953783347192
C=1.0 0.5861250179468526
C=1.1 0.5850241370309297
C=1.2000000000000002 0.584115313559634
C=1.3000000000000003 0.5833249992930905
C=1.4000000000000001 0.5826414467822387
C=1.5000000000000002 0.581980407461795
C=1.6 0.5813965004247106
C=1.7000000000000002 0.5808863817392089
C=1.8000000000000003 0.5804920821301753
C=1.9000000000000001 0.5801652903260501
C=2.0 0.5798514078202511
C=2.1 0.5795555463389299
C=2.2 0.5792894608626362
C=2.3000000000000003 0.5790628014477215
C=2.4000000000000004 0.5788595342946696
C=2.5000000000000004 0.5786675564902478
C=2.6 0.5784947039425343
C=2.7 0.5783242251090924
C=2.8000000000000003 0.5781527825713699
C=2.9000000000000004 0.577971588034194
C=3.0000000000000004 0.5777242036061592
C=3.1 0.5776695126429112
C=

In [42]:
# create result SVC
tfidf_vec = TfidfVectorizer()
X_train =tfidf_vec.fit_transform(data_train[0])
# print(X_train[:10,...])
y_train = data_train[1].values
X_test = tfidf_vec.transform(data_test['text'])


svc = svm.SVC(kernel='rbf',gamma=1.9684210526315788,probability=True)
svc.fit(X_train,y_train)
predict = svc.predict_proba(X_test)
print(predict.shape)
data_test['y'] = predict[:,1]
data_test.drop(columns='text',inplace=True)
data_test.to_csv('result_lr.csv',index=False)

(500, 2)


In [ ]:
# testing
print('\n')
tfidf_vec = TfidfVectorizer()
X = tfidf_vec.fit_transform(data_train['text'])
y = data_train[1].values
CV = KFold(n_splits=5,shuffle=True,random_state=241)
# np.power(10,np.linspace(0,1,100))
# np.linspace(0.01,10,20)
for C in np.linspace(0.1,20,100):
    log_los_values = []
    for train_index, test_index in CV.split(X):
        lr = LogisticRegression(penalty='l2',C=C)
        lr.fit(X[train_index],y[train_index])
        predict = lr.predict_proba(X[test_index])
        log_los_values.append(log_loss(y[test_index],predict[:,1]))
        #print(cv_results['test_score'])
    print(C, np.mean(log_los_values))
# 2.511886431509581 0.4820568735263769
# 2.323232323232323 0.48158033118945165
# 5.474747474747475 0.46380617530954443
# 5.451451451451451 0.463805939803994
# 5.4535353535353535 0.4638059371066535
# 5.929292929292929 0.4621232668547865